# import

In [3]:
import json
import os
import re
from pprint import pprint

from soynlp.normalizer import repeat_normalize

In [4]:
def dump_jsonl(data, output_path, append=False):
    mode = "a+" if append else "w"
    with open(output_path, mode, encoding="utf-8") as f:
        for line in data:
            json_record = json.dumps(line, ensure_ascii=False)
            f.write(json_record + "\n")


def load_jsonl(input_path) -> list:
    data = []
    with open(input_path, "r", encoding="utf-8") as f:
        for line in f:
            data.append(json.loads(line.rstrip("\n|\r")))
    return data

In [5]:
os.chdir("/home/nsw0311/nas_storage")

vocab_size = 24000

# load data

In [4]:
def processing(text):
    text = re.sub(r" +", r" ", text.strip())
    text = re.sub(r"(.{8,}?)\1+", r"\1", text)
    text = re.sub(
        r"[^ ㄱ-ㅎㅏ-ㅣ가-힣A-Za-z0-9~!@#$%\^\&*\(\)_\+\-=\[\]{},\./<>\?]", r"", text
    )
    text = re.sub(r"http.+", r"[URL]", text)

    return text.strip()

In [5]:
data = load_jsonl("datasets/RG/dialog_data.json")

In [6]:
len(data), data[0]

(12753322, {'content': '<user1> 웅.. 서럽', 'talk_id': 'kakaotalk_000000'})

In [7]:
data[0]["content"][8:]

'웅.. 서럽'

In [8]:
def gen():
    for row in data:
        yield processing(row["content"][8:])

In [10]:
user_defined_symbols = [
    "[PAD]",
    "[UNK]",
    "[CLS]",
    "[SEP]",
    "[MASK]",
    "[BOS]",
    "[EOS]",
    "[TSEP]",
    "[NAME]",
    "[URL]",
]
user_defined_symbols += [f"[UNK{i}]" for i in range(10)]
unused_token_num = 100
unused_list = [f"[UNUSED{i}]" for i in range(100)]
whole_user_defined_symbols = user_defined_symbols + unused_list

pprint(whole_user_defined_symbols)

['[PAD]',
 '[UNK]',
 '[CLS]',
 '[SEP]',
 '[MASK]',
 '[BOS]',
 '[EOS]',
 '[TSEP]',
 '[NAME]',
 '[URL]',
 '[UNK0]',
 '[UNK1]',
 '[UNK2]',
 '[UNK3]',
 '[UNK4]',
 '[UNK5]',
 '[UNK6]',
 '[UNK7]',
 '[UNK8]',
 '[UNK9]',
 '[UNUSED0]',
 '[UNUSED1]',
 '[UNUSED2]',
 '[UNUSED3]',
 '[UNUSED4]',
 '[UNUSED5]',
 '[UNUSED6]',
 '[UNUSED7]',
 '[UNUSED8]',
 '[UNUSED9]',
 '[UNUSED10]',
 '[UNUSED11]',
 '[UNUSED12]',
 '[UNUSED13]',
 '[UNUSED14]',
 '[UNUSED15]',
 '[UNUSED16]',
 '[UNUSED17]',
 '[UNUSED18]',
 '[UNUSED19]',
 '[UNUSED20]',
 '[UNUSED21]',
 '[UNUSED22]',
 '[UNUSED23]',
 '[UNUSED24]',
 '[UNUSED25]',
 '[UNUSED26]',
 '[UNUSED27]',
 '[UNUSED28]',
 '[UNUSED29]',
 '[UNUSED30]',
 '[UNUSED31]',
 '[UNUSED32]',
 '[UNUSED33]',
 '[UNUSED34]',
 '[UNUSED35]',
 '[UNUSED36]',
 '[UNUSED37]',
 '[UNUSED38]',
 '[UNUSED39]',
 '[UNUSED40]',
 '[UNUSED41]',
 '[UNUSED42]',
 '[UNUSED43]',
 '[UNUSED44]',
 '[UNUSED45]',
 '[UNUSED46]',
 '[UNUSED47]',
 '[UNUSED48]',
 '[UNUSED49]',
 '[UNUSED50]',
 '[UNUSED51]',
 '[UNUSED52]',
 '

# train

In [11]:
from tokenizers import Tokenizer
from tokenizers.models import WordPiece

bert_tokenizer = Tokenizer(WordPiece(unk_token="[UNK]"))

In [13]:
from tokenizers import normalizers
from tokenizers.normalizers import Lowercase, NFD, StripAccents

bert_tokenizer.normalizer = normalizers.Sequence([NFD(), Lowercase(), StripAccents()])

In [14]:
from tokenizers.pre_tokenizers import Whitespace

bert_tokenizer.pre_tokenizer = Whitespace()

In [15]:
from tokenizers.processors import TemplateProcessing

bert_tokenizer.post_processor = TemplateProcessing(
    single="[CLS] $A [SEP]",
    pair="[CLS] $A [SEP] $B:1 [SEP]:1",
    special_tokens=[(t, i) for i, t in enumerate(user_defined_symbols)],
)

In [16]:
from tokenizers.trainers import WordPieceTrainer

trainer = WordPieceTrainer(
    vocab_size=vocab_size,
    special_tokens=whole_user_defined_symbols,
)
bert_tokenizer.train_from_iterator(gen(), trainer)

In [17]:
output = bert_tokenizer.encode("테스트용인데 잘 되는거같아?")
print(output.ids)

bert_tokenizer.decode(output.ids)

[2, 8541, 3472, 911, 614, 3972, 1459, 146, 3]


'테스트 ##용이 ##ᆫ데 잘 되는거 ##같아 ?'

In [18]:
from tokenizers import decoders

bert_tokenizer.decoder = decoders.WordPiece()
bert_tokenizer.decode(output.ids)

'테스트용인데 잘 되는거같아?'

# convert transformers tokenizer and save

In [1]:
from transformers import ElectraTokenizerFast


fast_tokenizer = ElectraTokenizerFast(tokenizer_object=bert_tokenizer)

NameError: name 'bert_tokenizer' is not defined

In [20]:
fast_tokenizer.pad_token = "[PAD]"
fast_tokenizer.unk_token = "[UNK]"
fast_tokenizer.cls_token = "[CLS]"
fast_tokenizer.sep_token = "[SEP]"
fast_tokenizer.mask_token = "[MASK]"
fast_tokenizer.bos_token = "[BOS]"
fast_tokenizer.eos_token = "[EOS]"

special_tokens_dict = {"additional_special_tokens": user_defined_symbols}
fast_tokenizer.add_special_tokens(special_tokens_dict)

0

In [21]:
fast_tokenizer.decode(fast_tokenizer.encode("테스트용"))

2022-02-28 10:46:05.244813: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


'[CLS] 테스트용 [SEP]'

In [22]:
fast_tokenizer.save_pretrained("etc/DialogWordPiece")

('etc/DialogWordPiece/tokenizer_config.json',
 'etc/DialogWordPiece/special_tokens_map.json',
 'etc/DialogWordPiece/vocab.txt',
 'etc/DialogWordPiece/added_tokens.json',
 'etc/DialogWordPiece/tokenizer.json')

In [8]:
t = ElectraTokenizerFast.from_pretrained("etc/DialogWordPiece")

t(
    [["테스트용", "잘 되나"]],
    max_length=10,
    padding=True,
)

/home/nsw0311/.pyenv/versions/3.9.7/envs/main/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:2267: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


{'input_ids': [[2, 8541, 729, 3, 614, 4233, 3]], 'token_type_ids': [[0, 0, 0, 0, 1, 1, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1]]}